# Tarea 3

- Gestión de Operaciones — IN4273-2

- Matías Ignacio Saavera Guerra

# Pregunta 1 — Newsvendor

Dentro de la industria de la distribución de frutas, las paltas son uno de los productos más demandados, sobre todo las de la marca “VerdeVida”. Esta empresa distribuidora trabaja con productores de paltas que confían en “VerdeVida” para llevar su producto a los clientes finales. El objetivo de la empresa es asegurarse de contar con suficiente material de embalaje especializado para transportar las paltas en cajas premium.

Cada caja premium incluye 40 paltas y utiliza un embalaje especializado que ayuda a proteger la fruta durante el transporte. Un aspecto crítico es que las paltas tienen una rápida maduración y no pueden almacenarse por mucho tiempo; si la empresa no tiene suficiente embalaje para enviar las paltas, estas deben desecharse, lo que genera un costo significativo. Como parte del acuerdo comercial con los productores, “VerdeVida” debe pagar el costo por cada palta que no se logre transportar, lo que hace que minimizar los costos sea esencial para el negocio.

En este contexto, usted es el encargado de la distribución y debe satisfacer la demanda por transporte de paltas de los productores. Considere que esta demanda sigue una distribución normal con una media de 1.8 millones de paltas y una desviación estándar de 150,000 paltas. Además, usted sabe que el costo por cada palta que no se transporte es de 1.75 USD (según el acuerdo comercial mencionado anteriormente) y el costo del embalaje por caja es de 0.70 USD.

El problema que enfrenta “VerdeVida” es tener suficiente embalaje para las paltas que debe distribuir, ya que la empresa paga por cada palta no transportada.

---

## Preliminares

Para resolver este problema, se utilizará el modelo de Newsvendor, el cual busca minimizar el costo total esperado, el cual se compone de dos partes: el costo de exceso y el costo de faltante. El costo de exceso se refiere al costo de embalaje de las cajas que no se utilizan, mientras que el costo de faltante se refiere al costo de las paltas que no se logran transportar.

### Importación de librerías

En primer lugar, importamos las librerías necesarias para el desarrollo de la tarea.

In [4]:
import numpy as np
from scipy.stats import norm
import math

### Definición de constantes

Definimos las constantes que se utilizarán en el desarrollo de la tarea.

In [5]:
# Constants
MEAN_DEMAND = 1_800_000  # Mean demand in avocados
STD_DEMAND = 150_000     # Standard deviation of demand
BOX_CAPACITY = 40        # Avocados per box
BOX_COST = 0.70         # Cost per box in USD
AVOCADO_COST = 1.75     # Cost per lost avocado in USD
FILL_RATE_TARGET = 0.97  # Required fill rate for part d
NEW_AVOCADO_COST = 2.50 # New avocado cost for part f

### Definición de funciones

Definimos las funciones que se utilizarán en el desarrollo de la tarea.

In [6]:
def calculate_costs():
    """
    Calculate overage and underage costs
    Returns:
        tuple: (co, cu) where co is overage cost and cu is underage cost
    """
    # Overage cost (co) is the cost of ordering one too many boxes
    co = BOX_COST
    
    # Underage cost (cu) is the cost of not having a box when needed
    # One box holds 40 avocados, so missing one box costs 40 * avocado_cost
    cu = BOX_CAPACITY * AVOCADO_COST
    
    return co, cu

def optimal_order(mean, std, co, cu):
    """
    Calculate optimal order quantity using newsvendor formula
    Args:
        mean (float): Mean demand
        std (float): Standard deviation of demand
        co (float): Overage cost
        cu (float): Underage cost
    Returns:
        float: Optimal order quantity
    """
    z = norm.ppf(cu/(co + cu))
    return mean + z * std

def expected_cost(Q, mean, std, co, cu):
    """
    Calculate expected total cost for a given order quantity
    Args:
        Q (float): Order quantity
        mean (float): Mean demand
        std (float): Standard deviation of demand
        co (float): Overage cost
        cu (float): Underage cost
    Returns:
        float: Expected total cost
    """
    z = (Q - mean) / std
    L = std * (norm.pdf(z) + z * norm.cdf(z))
    return co * (Q - mean) + (co + cu) * (L)

def expected_lost_sales(Q, mean, std):
    """
    Calculate expected lost sales (shortages)
    Args:
        Q (float): Order quantity
        mean (float): Mean demand
        std (float): Standard deviation of demand
    Returns:
        float: Expected lost sales
    """
    z = (Q - mean) / std
    return std * (norm.pdf(z) - z * (1 - norm.cdf(z)))

def order_for_fill_rate(mean, std, fill_rate):
    """
    Calculate order quantity needed to achieve target fill rate
    Args:
        mean (float): Mean demand
        std (float): Standard deviation of demand
        fill_rate (float): Target fill rate
    Returns:
        float: Required order quantity
    """
    z = norm.ppf(fill_rate)
    return mean + z * std

---

## a) (1 pto.) Calcule los costos de overage y underage asociados al embalaje para las paltas. Justifique el cálculo de estos costos.

In [7]:
# Section A: Calculate overage and underage costs
print("\nSection A: Overage and Underage Costs")
print("-" * 50)
co, cu = calculate_costs()
print(f"Overage cost (co) = ${co:.2f} per box")
print(f"Underage cost (cu) = ${cu:.2f} per box")
print(f"Critical ratio (cu/(co+cu)) = {cu/(co+cu):.4f}")


Section A: Overage and Underage Costs
--------------------------------------------------
Overage cost (co) = $0.70 per box
Underage cost (cu) = $70.00 per box
Critical ratio (cu/(co+cu)) = 0.9901


### Contexto y Resultados

En la Sección A, se presentan los costos de overage y underage asociados al embalaje de las paltas, así como la razón crítica derivada de estos costos. Los resultados obtenidos son los siguientes:

- **Costo de overage (co)**: $0.70 por caja
- **Costo de underage (cu)**: $70.00 por caja
- **Razón crítica (cu/(co+cu))**: 0.9901

### Justificación del Cálculo de los Costos

#### Costo de Overage (co)

El costo de overage se refiere al costo incurrido por tener un exceso de inventario, es decir, cuando se ordenan más cajas de las necesarias y estas no se venden. Este costo incluye factores como el almacenamiento adicional, el deterioro del producto y la posible obsolescencia. En este caso, el costo de overage es de $0.70 por caja, lo cual refleja los gastos asociados a mantener inventario excedente.

#### Costo de Underage (cu)

El costo de underage se refiere al costo incurrido por tener un déficit de inventario, es decir, cuando se ordenan menos cajas de las necesarias y se pierde la oportunidad de venta. Este costo incluye la pérdida de ingresos, la insatisfacción del cliente y el posible impacto negativo en la reputación de la empresa. En este caso, el costo de underage es significativamente mayor, $70.00 por caja, lo cual subraya la importancia de evitar la falta de stock y las consecuencias económicas asociadas.

#### Razón Crítica

La razón crítica se calcula como la proporción del costo de underage sobre la suma de los costos de overage y underage:

$$ \text{Razón crítica} = \frac{cu}{co + cu} = \frac{70.00}{0.70 + 70.00} = 0.9901 $$

Esta razón crítica de 0.9901 indica que el costo de no tener suficiente inventario es extremadamente alto en comparación con el costo de tener un exceso de inventario. Por lo tanto, la estrategia óptima en este contexto sería priorizar la disponibilidad del producto para minimizar el riesgo de underage, incluso si esto implica incurrir en costos de overage.

### Conclusión

El análisis de los costos de overage y underage es fundamental para la toma de decisiones en la gestión de inventarios. En el caso del embalaje de las paltas, el alto costo de underage en comparación con el costo de overage sugiere que es preferible mantener un inventario ligeramente superior para evitar la pérdida de ventas y la insatisfacción del cliente. La razón crítica calculada refuerza esta estrategia, destacando la importancia de asegurar la disponibilidad del producto en el mercado.

---

## b) (1 pto.) Calcule el pedido óptimo de embalajes que maximiza las utilidades, considerando la distribución normal de la demanda. Además, calcule el costo esperado, incluyendo tanto el costo del embalaje como el de las paltas desechadas.

In [8]:
# Section B: Calculate optimal order and expected cost
print("\nSection B: Optimal Order and Expected Cost")
print("-" * 50)
# Convert avocado demand to box demand
mean_boxes = MEAN_DEMAND / BOX_CAPACITY
std_boxes = STD_DEMAND / BOX_CAPACITY

optimal_boxes = optimal_order(mean_boxes, std_boxes, co, cu)
expected_total_cost = expected_cost(optimal_boxes, mean_boxes, std_boxes, co, cu)

print(f"Optimal order: {optimal_boxes:.0f} boxes ({optimal_boxes*BOX_CAPACITY:,.0f} avocados)")
print(f"Expected total cost: ${expected_total_cost:,.2f}")


Section B: Optimal Order and Expected Cost
--------------------------------------------------
Optimal order: 53738 boxes (2,149,512 avocados)
Expected total cost: $624,767.21


### Contexto y Resultados

En la Sección B, se presentan los resultados del cálculo del pedido óptimo de embalajes y el costo total esperado. Los resultados obtenidos son los siguientes:

- **Pedido óptimo**: 53,738 cajas (2,149,512 paltas)
- **Costo total esperado**: $624,767.21

### Cálculo del Pedido Óptimo

El pedido óptimo se determina utilizando el modelo Newsvendor, que maximiza las utilidades al equilibrar los costos de overage y underage. En este caso, la demanda de paltas se asume que sigue una distribución normal, lo cual permite calcular el pedido óptimo basado en la razón crítica y la desviación estándar de la demanda.

#### Fórmula del Pedido Óptimo

El pedido óptimo $ Q^* $ se calcula utilizando la siguiente fórmula:

$$ Q^* = \mu + z \sigma $$

donde:
- $ \mu $ es la media de la demanda.
- $ \sigma $ es la desviación estándar de la demanda.
- $ z $ es el valor crítico correspondiente a la razón crítica en la distribución normal estándar.

### Cálculo del Costo Esperado

El costo total esperado incluye tanto el costo del embalaje como el costo de las paltas desechadas debido a overage. Este costo se calcula considerando los siguientes componentes:

1. **Costo del Embalaje**: El costo asociado a la cantidad de cajas pedidas.
2. **Costo de las Paltas Desechadas**: El costo de las paltas que no se venden y se desechan debido a un exceso de inventario.

#### Fórmula del Costo Esperado

El costo total esperado $ C $ se calcula como:

$$ C = C_{\text{embalaje}} + C_{\text{desecho}} $$

donde:
- $ C_{\text{embalaje}} $ es el costo del embalaje.
- $ C_{\text{desecho}} $ es el costo de las paltas desechadas.

### Justificación de los Resultados

El pedido óptimo de 53,738 cajas se ha calculado para maximizar las utilidades, minimizando los costos asociados tanto al exceso como a la falta de inventario. El costo total esperado de $624,767.21 refleja una combinación de los costos de embalaje y los costos de desecho, asegurando que la empresa mantenga un equilibrio entre la disponibilidad del producto y la minimización de desperdicios.

### Conclusión

El cálculo del pedido óptimo y del costo total esperado es esencial para la gestión eficiente de inventarios en el modelo Newsvendor. Al considerar la distribución normal de la demanda y los costos asociados, se puede determinar una estrategia óptima que maximice las utilidades y minimice los costos, asegurando la sostenibilidad y rentabilidad de la operación.

---

## c) (1 pto.) Suponga que se decide ordenar 45,000 cajas de embalaje. Calcule el número esperado de paltas que se perderían por falta de embalajes.

In [9]:
# Section C: Calculate expected lost sales for 45,000 boxes
print("\nSection C: Expected Lost Sales for 45,000 boxes")
print("-" * 50)
fixed_order = 45000
expected_losses = expected_lost_sales(fixed_order, mean_boxes, std_boxes)
print(f"Expected lost sales: {expected_losses*BOX_CAPACITY:,.0f} avocados")


Section C: Expected Lost Sales for 45,000 boxes
--------------------------------------------------
Expected lost sales: 59,841 avocados


### Contexto y Resultados

En la Sección C, se presentan los resultados del cálculo del número esperado de paltas que se perderían debido a la falta de embalajes al ordenar 45,000 cajas. Los resultados obtenidos son los siguientes:

- **Ventas perdidas esperadas**: 59,841 paltas

### Análisis del Cálculo

El cálculo del número esperado de paltas que se perderían por falta de embalajes se basa en la diferencia entre la demanda esperada y la cantidad de embalajes disponibles. En este caso, se asume que la demanda de paltas sigue una distribución normal, lo cual permite estimar las ventas perdidas utilizando técnicas estadísticas.

#### Fórmula de Ventas Perdidas Esperadas

El número esperado de ventas perdidas $ L $ se puede calcular utilizando la siguiente fórmula:

$$ L = \int_{Q}^{\infty} (x - Q) f(x) \, dx $$

donde:
- $ Q $ es la cantidad de cajas ordenadas (45,000 cajas).
- $ f(x) $ es la función de densidad de probabilidad de la demanda de paltas.

### Justificación de los Resultados

El resultado de 59,841 paltas perdidas se deriva de la diferencia entre la demanda esperada y la cantidad de embalajes disponibles. Al ordenar 45,000 cajas, la empresa no puede satisfacer completamente la demanda, lo que resulta en una pérdida de ventas. Este cálculo es crucial para entender el impacto de no ordenar la cantidad óptima de embalajes y para tomar decisiones informadas sobre la gestión de inventarios.

### Factores a Considerar

1. **Demanda Estacional**: La demanda de paltas puede variar estacionalmente, lo cual debe ser considerado al calcular las ventas perdidas.
2. **Elasticidad de la Demanda**: La sensibilidad de la demanda a la disponibilidad del producto puede afectar el número de ventas perdidas.
3. **Costos de Oportunidad**: Las ventas perdidas representan un costo de oportunidad significativo, ya que la empresa pierde ingresos potenciales y puede afectar la satisfacción del cliente.

### Conclusión

El cálculo del número esperado de paltas que se perderían por falta de embalajes al ordenar 45,000 cajas es esencial para la toma de decisiones en la gestión de inventarios. Los resultados indican que se perderían

---

## d) (1 pto.) Calcule el pedido que asegura un Fill Rate del 97%. ¿Cuál sería la utilidad esperada en este caso?

In [10]:
# Section D: Calculate order for 97% fill rate
print("\nSection D: Order for 97% Fill Rate")
print("-" * 50)
fill_rate_order = order_for_fill_rate(mean_boxes, std_boxes, FILL_RATE_TARGET)
fill_rate_cost = expected_cost(fill_rate_order, mean_boxes, std_boxes, co, cu)
print(f"Order quantity for {FILL_RATE_TARGET*100}% fill rate: {fill_rate_order:.0f} boxes")
print(f"Expected cost with fill rate target: ${fill_rate_cost:,.2f}")


Section D: Order for 97% Fill Rate
--------------------------------------------------
Order quantity for 97.0% fill rate: 52053 boxes
Expected cost with fill rate target: $506,662.75


### Contexto y Resultados

En la Sección D, se presentan los resultados del cálculo del pedido necesario para asegurar un Fill Rate del 97% y el costo esperado asociado a este objetivo. Los resultados obtenidos son los siguientes:

- **Cantidad de pedido para un Fill Rate del 97%**: 52,053 cajas
- **Costo esperado con el objetivo de Fill Rate**: $506,662.75

### Análisis del Cálculo

El Fill Rate es una métrica que mide el porcentaje de demanda que se satisface directamente del inventario disponible. Un Fill Rate del 97% indica que el 97% de la demanda se puede cumplir sin incurrir en ventas perdidas. Para calcular la cantidad de pedido que asegura este Fill Rate, se utiliza la distribución de la demanda y se determina el punto en el cual el 97% de la demanda se encuentra cubierta.

#### Fórmula del Fill Rate

El Fill Rate $ FR $ se puede calcular utilizando la siguiente fórmula:

$$ FR = 1 - \frac{E[L(Q)]}{\mu} $$

donde:
- $ E[L(Q)] $ es el número esperado de ventas perdidas.
- $ \mu $ es la media de la demanda.

Para asegurar un Fill Rate del 97%, se ajusta la cantidad de pedido $ Q $ de manera que $ FR = 0.97 $.

### Cálculo de la Utilidad Esperada

La utilidad esperada se calcula como la diferencia entre los ingresos esperados y los costos totales esperados. Los costos totales incluyen tanto el costo del embalaje como el costo de las paltas desechadas debido a overage.

#### Fórmula de la Utilidad Esperada

La utilidad esperada $ U $ se puede calcular utilizando la siguiente fórmula:

$$ U = I - C $$

donde:
- $ I $ son los ingresos esperados.
- $ C $ es el costo total esperado.

### Justificación de los Resultados

El pedido de 52,053 cajas asegura un Fill Rate del 97%, lo cual minimiza las ventas perdidas y maximiza la satisfacción del cliente. El costo esperado de $506,662.75 refleja los gastos asociados a mantener este nivel de servicio. La utilidad esperada se puede calcular restando este costo de los ingresos esperados, considerando el precio de venta de las paltas y la cantidad vendida.

### Conclusión

El cálculo del pedido que asegura un Fill Rate del 97% es crucial para la gestión eficiente de inventarios y la maximización de utilidades. Al ordenar 52,053 cajas, la empresa puede satisfacer el 97% de la demanda, minimizando las ventas perdidas y optimizando la satisfacción del cliente. La utilidad esperada en este caso se obtiene restando el costo total esperado de los ingresos esperados, proporcionando una medida clara de la rentabilidad de la operación.

---

## e) (1 pto.) Explique cómo varía el pedido óptimo si la empresa decide priorizar la reducción de las paltas desechadas frente a la minimización de costos de exceso de embalajes. ¿Cómo influiría esto en la utilidad esperada y qué factores deberían considerarse para tomar una decisión bajo incertidumbre?

### Contexto y Resultados

El análisis de la priorización de la reducción de residuos se fundamenta en los resultados obtenidos en los bloques anteriores, los cuales indican una tendencia significativa hacia la acumulación de inventario excedente al priorizar la reducción de residuos. Este fenómeno se debe a que la cantidad de pedido óptima se desplaza hacia la cola derecha de la distribución de la demanda, incrementando así la probabilidad de excedentes.

### Factores a considerar:

1. **Impacto ambiental de los residuos**: La reducción de residuos tiene un efecto positivo en la disminución de la huella ecológica de la empresa. La minimización de desechos contribuye a la sostenibilidad ambiental y al cumplimiento de normativas ecológicas.

2. **Costos de reputación**: La gestión eficiente de residuos mejora la imagen corporativa y fortalece la reputación de la empresa ante consumidores y stakeholders. Una empresa comprometida con la sostenibilidad puede obtener ventajas competitivas en el mercado.

3. **Relación a largo plazo con los proveedores**: La priorización de la reducción de residuos puede requerir ajustes en las relaciones con los proveedores, incluyendo negociaciones para la entrega de materiales en cantidades más precisas y la implementación de prácticas de producción más sostenibles.

4. **Capacidad y costos de almacenamiento**: Un aumento en la cantidad de pedido puede llevar a mayores requerimientos de espacio de almacenamiento, lo cual implica costos adicionales. Es crucial evaluar la capacidad de almacenamiento disponible y los costos asociados para evitar ineficiencias logísticas.

5. **Requisitos de capital de trabajo**: La gestión de inventarios más grandes requiere una mayor inversión en capital de trabajo. Es necesario analizar la disponibilidad de recursos financieros y su impacto en la liquidez de la empresa.

### Conclusión

En conclusión, la priorización de la reducción de residuos presenta tanto ventajas como desafíos. Es esencial realizar un análisis exhaustivo de los factores mencionados para tomar decisiones informadas que equilibren la sostenibilidad ambiental con la eficiencia operativa y financiera.

---

## f) (1 pto.) Si el costo por palta aumenta a 2.50 USD debido a una menor producción en la temporada, ¿cómo afectaría este cambio la cantidad óptima de embalajes a pedir y los costos asociados? Reflexione sobre cómo los cambios en los costos de la materia prima pueden influir en la toma de decisiones en el modelo Newsvendor.

In [12]:
# Section F: Recalculate with new avocado cost
print("\nSection F: Analysis with New Avocado Cost")
print("-" * 50)
# Recalculate costs with new avocado cost
new_co = co
new_cu = BOX_CAPACITY * NEW_AVOCADO_COST

new_optimal_boxes = optimal_order(mean_boxes, std_boxes, new_co, new_cu)
new_expected_cost = expected_cost(new_optimal_boxes, mean_boxes, std_boxes, new_co, new_cu)

print(f"New optimal order: {new_optimal_boxes:.0f} boxes ({new_optimal_boxes*BOX_CAPACITY:,.0f} avocados)")
print(f"New expected total cost: ${new_expected_cost:,.2f}")
print(f"Change in optimal order: {new_optimal_boxes - optimal_boxes:.0f} boxes")
print(f"Percentage increase in order: {((new_optimal_boxes/optimal_boxes) - 1)*100:.1f}%")


Section F: Analysis with New Avocado Cost
--------------------------------------------------
New optimal order: 54224 boxes (2,168,965 avocados)
New expected total cost: $936,183.52
Change in optimal order: 486 boxes
Percentage increase in order: 0.9%


### Reflexión sobre la influencia de los cambios en los costos de la materia prima en la toma de decisiones en el modelo Newsvendor

La variabilidad en los costos de la materia prima es un factor crítico que puede afectar significativamente la toma de decisiones en el modelo Newsvendor. Este modelo, utilizado para determinar la cantidad óptima de pedido que minimiza los costos totales, es altamente sensible a las fluctuaciones en los precios de los insumos.

#### Contexto y Resultados

En el análisis reciente, se observó un cambio en el costo de los aguacates, lo cual impactó directamente en la cantidad óptima de pedido y en los costos esperados totales. Los resultados obtenidos fueron los siguientes:

- **Nuevo pedido óptimo**: 54,224 cajas (2,168,965 aguacates)
- **Nuevo costo total esperado**: $936,183.52
- **Cambio en el pedido óptimo**: 486 cajas
- **Incremento porcentual en el pedido**: 0.9%

#### Análisis de los Resultados

El incremento en el costo de la materia prima llevó a un ajuste en la cantidad óptima de pedido, aumentando en 486 cajas, lo que representa un incremento del 0.9%. Este ajuste es una respuesta directa a la necesidad de equilibrar los costos de adquisición con los costos de mantener inventario y los costos de oportunidad asociados con la falta de stock.

#### Factores a Considerar

1. **Elasticidad de la demanda**: La sensibilidad de la demanda a los cambios en el precio de la materia prima puede influir en la cantidad óptima de pedido. En mercados donde la demanda es altamente elástica, un aumento en los costos de la materia prima puede llevar a una reducción significativa en la cantidad demandada.

2. **Costos de almacenamiento**: Un aumento en la cantidad de pedido puede incrementar los costos de almacenamiento. Es crucial evaluar la capacidad de almacenamiento disponible y los costos adicionales que esto pueda implicar.

3. **Riesgo de obsolescencia**: En productos perecederos como los aguacates, un aumento en la cantidad de pedido puede incrementar el riesgo de obsolescencia y desperdicio, lo cual debe ser considerado en la toma de decisiones.

4. **Capital de trabajo**: La necesidad de financiar un mayor inventario puede requerir una mayor inversión en capital de trabajo, afectando la liquidez de la empresa.

5. **Relaciones con proveedores**: Cambios en los costos de la materia prima pueden requerir renegociaciones con los proveedores para asegurar precios más favorables o condiciones de pago más flexibles.

#### Conclusión

Los cambios en los costos de la materia prima tienen un impacto significativo en la toma de decisiones dentro del modelo Newsvendor. Es esencial realizar un análisis detallado de los factores mencionados para ajustar la cantidad óptima de pedido de manera que se minimicen los costos totales y se maximice la eficiencia operativa. La capacidad de adaptarse rápidamente a las fluctuaciones en los precios de los insumos es crucial para mantener la competitividad y la rentabilidad en mercados dinámicos.